In [21]:
# Instalação e setup
%pip -q install google-genai google-adk

# Imports
import os, textwrap, warnings
from google.colab import userdata
from datetime import date
from IPython.display import display, Markdown
from google import genai
from google.genai import types
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search

warnings.filterwarnings("ignore")
os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")

client = genai.Client()
MODEL_ID = "gemini-2.0-flash"

# Funções auxiliares
def call_agent(agent: Agent, message_text: str) -> str:
    session_service = InMemorySessionService()
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    content = types.Content(role="user", parts=[types.Part(text=message_text)])
    final_response = ""
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
            for part in event.content.parts:
                if part.text is not None:
                    final_response += part.text + "\n"
    return final_response

def to_markdown(text):
    text = text.replace('•', '  *')
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

# Agente 1 – Buscador (filtra eventos futuros)
def agente_buscador_oportunidades(perfil_usuario, data_de_hoje):
    buscador = Agent(
        name="agente_buscador_oportunidades",
        model=MODEL_ID,
        instruction="""
        Você é um buscador especializado em encontrar oportunidades de crescimento pessoal e profissional na internet. Sua resposta deverá ser amigável com utilização moderada de emojis.

        Utilize apenas uma frase em sua introdução. Evite excesso de texto.

        Com base no perfil do usuário e na data atual, use a ferramenta google_search para buscar **eventos, cursos, bolsas, mentorias ou vagas cujas datas de realização ou inscrição sejam HOJE ou no FUTURO**.

        Ignore completamente eventos já encerrados ou com inscrições vencidas. Priorize conteúdos com:
        - Data de realização futura
        - Prazos de inscrição ainda abertos
        - Indicação clara de 'Inscrições abertas', 'Participe agora', 'Vem aí', 'Em breve', 'Inscreva-se até XX/XX'

        Busque no máximo 5 oportunidades que sejam REAIS, ATUAIS e RELEVANTES para o perfil informado.
        """,
        description="Agente que busca oportunidades atuais e futuras",
        tools=[google_search]
    )
    entrada = f"Perfil do usuário: {perfil_usuario}\nHoje é: {data_de_hoje}. Busque apenas oportunidades com prazos abertos ou datas futuras."
    return call_agent(buscador, entrada)

# Agente 2 – Classificador (tabela organizada)
def agente_classificador(oportunidades_encontradas):
    classificador = Agent(
        name="agente_classificador",
        model=MODEL_ID,
        instruction="""
        Você é um analista de oportunidades.

        Classifique cada item da lista abaixo e organize em uma tabela com as colunas:
        Título | Tipo (curso, evento, vaga, etc.) | Área (ex: tecnologia, design, etc.) | Urgência (baixo, médio, alto) | Link

        Urgência:
        - Alto: prazos nos próximos 5 dias
        - Médio: prazos entre 6 e 15 dias
        - Baixo: mais de 15 dias ou sem prazo definido

        Deixe a apresentação clara, com texto direto e sem rodeios.
        """,
        description="Agente que organiza oportunidades em tabela"
    )
    return call_agent(classificador, f"Oportunidades encontradas:\n{oportunidades_encontradas}")

# Agente 3 – Planejador (dividido por semana)
def agente_planejador(perfil_usuario, oportunidades_classificadas):
    planejador = Agent(
        name="agente_planejador",
        model=MODEL_ID,
        instruction="""
        Você é um planejador estratégico.

        Com base no perfil do usuário e nas oportunidades listadas, crie um plano de ação para os próximos 30 dias, dividido por semanas.

        Para cada semana, indique:
        - Quais oportunidades focar
        - Quais ações devem ser feitas (ex: enviar inscrição, assistir evento, revisar portfólio)
        - Uma dica breve e motivadora

        Use listas organizadas por semana, com tom direto e encorajador.
        """,
        description="Agente que sugere plano de ação semanal"
    )
    entrada = f"Perfil do usuário: {perfil_usuario}\nOportunidades classificadas:\n{oportunidades_classificadas}"
    return call_agent(planejador, entrada)

# Agente 4 – Coach (minimalista e elegante)
def agente_coach(perfil_usuario, plano_acao):
    coach = Agent(
        name="agente_coach",
        model=MODEL_ID,
        instruction="""
        Você é um coach virtual com tom elegante, direto e inspirador.

        Apresente o plano de ação do usuário com leveza, encorajamento e clareza.
        Use emojis com moderação e apenas quando fizerem sentido. Evite excesso de texto.

        O objetivo é deixar o usuário animado e confiante para começar. Seja breve e impactante.
        """,
        description="Coach motivacional com estilo minimalista"
    )
    entrada = f"Perfil do usuário: {perfil_usuario}\nPlano de ação:\n{plano_acao}"
    return call_agent(coach, entrada)

# Execução do sistema completo
print("🚀 Bem-vindo ao OPORTU.AI – seu curador de oportunidades de crescimento!")
perfil_usuario = input("🧑‍🎓 Descreva seu perfil e interesses (ex: 'sou estudante de design e busco estágio e eventos'): ")
data_de_hoje = date.today().strftime("%d/%m/%Y")

if not perfil_usuario:
    print("❗ Você precisa descrever seu perfil.")
else:
    print("\n🔎 Buscando oportunidades...")
    oportunidades = agente_buscador_oportunidades(perfil_usuario, data_de_hoje)
    display(Markdown("### 🎯 Oportunidades encontradas:"))
    display(to_markdown(oportunidades))

    print("\n📂 Classificando oportunidades...")
    oportunidades_classificadas = agente_classificador(oportunidades)
    display(Markdown("### 🗂️ Oportunidades classificadas:"))
    display(to_markdown(oportunidades_classificadas))

    print("\n🗓️ Criando plano de ação...")
    plano = agente_planejador(perfil_usuario, oportunidades_classificadas)
    display(Markdown("### 🛠️ Plano de ação sugerido:"))
    display(to_markdown(plano))

    print("\n💬 Preparando mensagem motivacional...")
    mensagem_final = agente_coach(perfil_usuario, plano)
    display(Markdown("### 💡 Coach Virtual diz:"))
    display(to_markdown(mensagem_final))


🚀 Bem-vindo ao OPORTU.AI – seu curador de oportunidades de crescimento!
🧑‍🎓 Descreva seu perfil e interesses (ex: 'sou estudante de design e busco estágio e eventos'): Procuro oportunidades de UX Design

🔎 Buscando oportunidades...


### 🎯 Oportunidades encontradas:

> Olá! 👋 Posso te ajudar a encontrar oportunidades de UX Design que estão com inscrições abertas ou que acontecerão em breve. 😊
> 
> 
> Encontrei algumas oportunidades que podem te interessar! 😉
> 
> 1.  **Conferências de UX Design em 2025:** A Interaction Design Foundation (IxDF) está organizando diversas conferências ao longo de 2025, tanto online quanto presenciais, com foco em temas como pesquisa em UX, design de interação e usabilidade. Fique de olho nas datas e locais para não perder nenhuma! 🗓️
> 
> 2.  **Vagas UX Design na VagasUX:** A VagasUX está divulgando diversas oportunidades remotas e presenciais para UX Designers, incluindo vagas para UI Designers, Product Designers e UX Writers. Há vagas para todos os níveis de experiência, desde estágio até sênior. 💼
> 
> 3.  **Curso de UX Design e Agilidade na PUC Minas:** A PUC Minas está com inscrições abertas até 30 de abril de 2025 para o curso de pós-graduação em UX Design e Agilidade. O curso aborda temas como inteligência artificial, projetos e tecnologias de UX, design de conteúdo e prototipação. 📚
> 
> 4.  **Mentoria Avançada em UX Design na Mergo:** A Mergo está com pré-inscrições abertas para a Sala VIP UX, um programa de desenvolvimento contínuo para profissionais de UX em nível pleno e sênior. O programa oferece mentorias individuais, encontros coletivos e acesso a conteúdos exclusivos. As inscrições terminam em 30 de maio de 2025 e as mentorias começam em 23 de junho de 2025. 🧑‍🏫
> 
> 5.  **Programa Desenvolve:** Estão abertas 20 mil vagas para curso 100% online e gratuito, com foco em introdução à tecnologia, inteligência artificial e análise de dados. As inscrições vão até 14 de abril! É só acessar o site Desenvolve.
> 



📂 Classificando oportunidades...


### 🗂️ Oportunidades classificadas:

...

### 🛠️ Plano de ação sugerido:

> ## Plano de Ação Estratégico (Próximos 30 Dias)
> 
> **Semana 1: Foco na Ação Rápida e Direcionada**
> 
> *   **Oportunidade:** Programa Desenvolve
> *   **Ações:**
>     *   Enviar inscrição para o Programa Desenvolve.
>     *   Reservar tempo na agenda para o curso, caso seja aceito.
> *   **Dica:** "Aja agora! Uma inscrição é o primeiro passo para um futuro cheio de oportunidades."
> 
> **Semana 2: Ampliando Horizontes e Explorando o Mercado**
> 
> *   **Oportunidade:** Vagas UX Design na VagasUX
> *   **Ações:**
>     *   Navegar na plataforma VagasUX e identificar 5 vagas que se encaixem no seu perfil.
>     *   Adaptar seu currículo e portfólio para destacar as habilidades mais relevantes para essas vagas.
> *   **Dica:** "O mercado está esperando por você. Mostre seu valor!"
> 
> **Semana 3: Investimento no Desenvolvimento Contínuo**
> 
> *   **Oportunidade:** Curso de UX Design e Agilidade na PUC Minas
> *   **Ações:**
>     *   Analisar a ementa do curso e verificar se os tópicos abordados complementam suas habilidades.
>     *   Se o curso for interessante, iniciar o processo de inscrição.
> *   **Dica:** "Aprender é crescer. Invista em você e veja seus horizontes se expandirem!"
> 
> **Semana 4: Networking e Preparação para o Futuro**
> 
> *   **Oportunidade:** Conferências de UX Design em 2025 e Mentoria Avançada em UX Design na Mergo
> *   **Ações:**
>     *   Pesquisar mais sobre a Mentoria Avançada na Mergo e agendar uma conversa inicial, se possível.
>     *   Começar a planejar a participação nas conferências de UX Design em 2025, definindo quais são mais relevantes para seus objetivos.
> *   **Dica:** "O futuro se constrói agora. Planeje, conecte-se e prepare-se para o sucesso!"
> 



💬 Preparando mensagem motivacional...


### 💡 Coach Virtual diz:

> 🚀 Plano de ação UX Design nos próximos 30 dias:
> 
> *   🎯 **Semana 1**: Inscreva-se no Programa Desenvolve. Aja rápido!
> *   💼 **Semana 2**: Explore VagasUX e adapte seu currículo. Mostre seu valor!
> *   📚 **Semana 3**: Invista no curso da PUC Minas. Aprender é crescer!
> *   🌐 **Semana 4**: Mentoria Mergo e planeje as conferências de 2025. Conecte-se!
> 
> Acredite no seu potencial e siga em frente! ✨
